# 7. 텍스트 데이터 다루기
# Working With Text Data

## 7.1 문자열 데이터 타입

데이터 분석을 하면서 문자열 데이터를 접할 수 있는 경우는 아래와 같이 네 가지 종류가 있다.
- 범주형 데이터
- 범주에 의미를 연결시킬 수 있는 임의의 문자열
- 구조화된 문자열 데이터
- 텍스트 데이터

__범주형 데이터__는 고정된 목록(개수)로 구성된다. 예를 들어 가장 좋아하는 색을 묻는 설문에서 `빨강, 녹색, 파랑`과 같이 하나를 선택하는 경우를 말한다.<br />
__범주에 의미를 연결시킬 수 있는 임의의 문자열은__ "사과 같이 빨간색", "불그스름한 색" 은 "빨강"이라는 범주에 연결시킬 수 있는 문자열을 의미한다. <br />
__구조화된 문자열 데이터__는 범주형 데이터 같은 데이터는 아니지만, 주소나 장소, 이메일, 전화번호 등 처럼 일정한 구조를 가진 데이터를 말한다. <br />
__텍스트 데이터__는 뉴스의 기사, 댓글, 논문 등과 같이 일반적인 텍스트 데이터를 말한다. 

텍스트 분석에서 데이터 셋을 ***말뭉치(corpus)***라 하고, 하나의 텍스트를 의미하는 각 데이터 포인트를 ***문서(document)***라고 한다. 

## 7.2 예제 애플리케이션: 영화 리뷰 감성 분석 (Sentiment Analysis)
이 교재에서는 텍스트 분석 예제로 스탠포드 대학교 연구원인 앤드류 마스가 IMDB(Internet Movie DataBase)에서 수집한 영화 리뷰 데이터셋을 사용한다. 해당 데이터 셋은 http://ai.stanford.edu/~amaas/data/sentiment/ 에서 다운 받을 수 있다. 다운로드 후 압축을 풀면 약 400MB 크기의 데이터가 있고 아래와 같이 구성되어 있다. IMDb 사이트에는 1 ~ 10까지 점수가 있다. 이 데이터셋은 7점이상은 positive, 4점 이하는 negative인 이진 분류 데이터셋으로 구분 되어있다.

In [1]:
## Windows10 - 64bit
!tree data/aclImdb

## Mac OS
# !tree -dL 2 data/aclImdb

새 볼륨 볼륨에 대한 폴더 경로의 목록입니다.
볼륨 일련 번호가 00000028 CC7F:4282입니다.
D:\USERS\CJH\DEV\STUDY\STUDY\ML_DL\INTRO_TO_ML_WITH_PYTHON\DATA\ACLIMDB
├─test
│  ├─neg
│  └─pos
└─train
    ├─neg
    └─pos


`pos`폴더에는 긍정적인(positive) 리뷰가 각각 하나의 파일로 나위어 있고, `neg` 폴더(negative)에도 마찬가지로 있다. `unsup` 폴더는 레이블이 없는 데이터를 담고 있는데, 실습에서는 필요하지 않으므로 삭제해준다. 

In [2]:
# Mac OS
!rm -r data/aclImdb/train/unsup

rm: cannot remove `data/aclImdb/train/unsup': No such file or directory


`scikit-learn`의 `load_files`함수를 사용해서 파일을 읽어 올 수 있다. `load_files`로 폴더의 데이터를 읽을 때 레이블은 폴더의 알파벳 순서에 따라 0부터 부여한다. 여기서의 데이터셋은 neg 폴더에는 레이블이 0이되고, pos는 1이 된다.

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mglearn
from IPython.display import display

In [4]:
from sklearn.datasets import load_files

%time reviews_train = load_files('./data/aclImdb/train/')

Wall time: 3.33 s


In [5]:
text_train, y_train = reviews_train.data, reviews_train.target
print('text_train의 타입 : {}'.format(type(text_train)))
print('text_train의 길이 : {}'.format(len(text_train)))
print('text_train[6]: \n{}'.format(text_train[6]))

text_train의 타입 : <class 'list'>
text_train의 길이 : 25000
text_train[6]: 
b"This movie has a special way of telling the story, at first i found it rather odd as it jumped through time and I had no idea whats happening.<br /><br />Anyway the story line was although simple, but still very real and touching. You met someone the first time, you fell in love completely, but broke up at last and promoted a deadly agony. Who hasn't go through this? but we will never forget this kind of pain in our life. <br /><br />I would say i am rather touched as two actor has shown great performance in showing the love between the characters. I just wish that the story could be a happy ending."


In [6]:
# html 태그 제거 - <br />태그
text_train = [doc.replace(b"<br />", b" ") for doc in text_train]

In [7]:
print('클래스별 샘플 수 (Training data): {}'.format(np.bincount(y_train)))

클래스별 샘플 수 (Training data): [12500 12500]


이번에는 같은 방법으로 Test data를 가져온다.

In [9]:
%%time
reviews_test = load_files("./data/aclImdb/test/")
text_test, y_test = reviews_test.data, reviews_test.target
print("테스트 데이터의 문서 수: {}".format(len(text_test)))
print("클래스별 샘플 수 (테스트 데이터): {}".format(np.bincount(y_test)))
text_test = [doc.replace(b"<br />", b" ") for doc in text_test]

테스트 데이터의 문서 수: 25000
클래스별 샘플 수 (테스트 데이터): [12500 12500]
Wall time: 1min 22s


## 7.3 텍스트 데이터를 BOW로 표현하기
머신러닝에서 텍스트를 표현하는 방법 중 ***BOW(Bag Of Words)*** 는 간단하면서도 효과적이어서 많이 사용한다. 하지만, BOW를 사용할 경우, 텍스트 데이터는 텍스트 자체의 문장, 문단과 같은 텍스트 구조는 사라지게 되며 단어의 빈도수만 확인 가능하다. BOW를 표현하기 위해서는 아래의 세 가지 단계가 필요하다. <br />
1. **토큰화(tokenization)**: 각 문서(문장)을 단어(토큰)으로 나누는 작업을 말한다.
2. **어휘 사전 구축**: 모든 문서에 나타난 모든 단어의 어휘를 모으고 번호를 매긴다(알파벳 순서)
3. **인코딩**: 어휘 사전의 단어가 문서마다 몇 번이나 나타나는지를 계산한다.

### 7.3.1 샘플 데이터에 BOW 적용하기

In [10]:
# 영화 리뷰 데이터가 아닌 임의의 데이터

bards_words = ["The fool doth think he is wise,",
              "but the wise man knows himself to be a fool"]

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect.fit(bards_words)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [12]:
print("어휘 사전의 크기: {}".format(len(vect.vocabulary_)))
print("어휘 사전의 내용:\n {}".format(vect.vocabulary_))

어휘 사전의 크기: 13
어휘 사전의 내용:
 {'man': 8, 'he': 4, 'himself': 5, 'wise': 12, 'doth': 2, 'fool': 3, 'be': 0, 'but': 1, 'is': 6, 'knows': 7, 'think': 10, 'the': 9, 'to': 11}


샘플데이터에 대해 BOW형태로 만들려면 `transform`메소드를 호출한다.

In [13]:
bag_of_words = vect.transform(bards_words)
print("BOW: {}".format(repr(bag_of_words)))

BOW: <2x13 sparse matrix of type '<class 'numpy.int64'>'
	with 16 stored elements in Compressed Sparse Row format>


In [14]:
print("BOW의 밀집 표현:\n{}".format(bag_of_words.toarray()))

BOW의 밀집 표현:
[[0 0 1 1 1 0 1 0 0 1 1 0 1]
 [1 1 0 1 0 1 0 1 1 1 0 1 1]]


### 7.3.2 영화 리뷰에 대한 BOW

In [15]:
%%time
vect = CountVectorizer().fit(text_train)
X_train = vect.transform(text_train)
print("X_train:\n{}".format(repr(X_train)))

X_train:
<25000x74849 sparse matrix of type '<class 'numpy.int64'>'
	with 3431196 stored elements in Compressed Sparse Row format>
Wall time: 11.1 s


`CounterVectorizer`객체의 `get_feature_name`메소드는 각 특성에 해당하는 단어를 리스트로 반환한다.

In [16]:
feature_names = vect.get_feature_names()
print('특성 개수: {}'.format(len(feature_names)))
print('처음 20개 특성:\n{}'.format(feature_names[:20]))
print("20,010에서 20,030까지 특성:\n{}".format(feature_names[20010:20030]))
print("매 2,000번쨰 특성:\n{}".format(feature_names[::2000]))

특성 개수: 74849
처음 20개 특성:
['00', '000', '0000000000001', '00001', '00015', '000s', '001', '003830', '006', '007', '0079', '0080', '0083', '0093638', '00am', '00pm', '00s', '01', '01pm', '02']
20,010에서 20,030까지 특성:
['dratted', 'draub', 'draught', 'draughts', 'draughtswoman', 'draw', 'drawback', 'drawbacks', 'drawer', 'drawers', 'drawing', 'drawings', 'drawl', 'drawled', 'drawling', 'drawn', 'draws', 'draza', 'dre', 'drea']
매 2,000번쨰 특성:
['00', 'aesir', 'aquarian', 'barking', 'blustering', 'bête', 'chicanery', 'condensing', 'cunning', 'detox', 'draper', 'enshrined', 'favorit', 'freezer', 'goldman', 'hasan', 'huitieme', 'intelligible', 'kantrowitz', 'lawful', 'maars', 'megalunged', 'mostey', 'norrland', 'padilla', 'pincher', 'promisingly', 'receptionist', 'rivals', 'schnaas', 'shunning', 'sparse', 'subset', 'temptations', 'treatises', 'unproven', 'walkman', 'xylophonist']


특성 추출을 위한 전처리를 적용하기 전에 현재의 BOW를 이용하여 분류기를 구현해보자.

In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

%time scores = cross_val_score(LogisticRegression(), X_train, y_train, cv=5)
print("CV Avg. Score: {:.2f}".format(np.mean(scores)))

Wall time: 32.7 s
CV Avg. Score: 0.88


`LogisticRegression`에 있는 파라미터 `C`를 조정하기 위해 그리드 서치를 이용해보자.

In [22]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
%time grid.fit(X_train, y_train)
print("최상의 CV 점수: {:.2f}".format(grid.best_score_))
print("최적의 파라미터: ", grid.best_params_)

Wall time: 1min 50s
최상의 CV 점수: 0.89
최적의 파라미터:  {'C': 0.1}


In [23]:
X_test = vect.transform(text_test)
print("테스트 점수: {:.2f}".format(grid.score(X_test, y_test)))

테스트 점수: 0.88


여기서 사용한 `CounterVectorizer`는 정규표현식을 사용해 토큰을 추출한다. 기본적으로 사용하는 정규표현식은 "\b\w\w+\b"이다. \b로 경계를 구분하고 \w로 적어도 둘 이상의 문자나 숫자가 연속된 단어를 찾아 토큰을 추출한다. <br />
`min_df` 파라미터로 토큰이 나타날 최소 문서 개수를 지정해 줄 수 있다 아래의 코드는 최소 문서 개수를 `min_df=5`로 설정해 주었다.

In [25]:
vect = CountVectorizer(min_df=5).fit(text_train)
X_train = vect.transform(text_train)
print("min_df로 제한한 X_train: {}".format(repr(X_train)))

min_df로 제한한 X_train: <25000x27271 sparse matrix of type '<class 'numpy.int64'>'
	with 3354014 stored elements in Compressed Sparse Row format>


In [26]:
feature_names = vect.get_feature_names()

print("First 50 features:\n{}".format(feature_names[:50]))
print("Features 20010 to 20030:\n{}".format(feature_names[20010:20030]))
print("Every 700th feature:\n{}".format(feature_names[::700]))

First 50 features:
['00', '000', '007', '00s', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '100', '1000', '100th', '101', '102', '103', '104', '105', '107', '108', '10s', '10th', '11', '110', '112', '116', '117', '11th', '12', '120', '12th', '13', '135', '13th', '14', '140', '14th', '15', '150', '15th', '16', '160', '1600', '16mm', '16s', '16th']
Features 20010 to 20030:
['repentance', 'repercussions', 'repertoire', 'repetition', 'repetitions', 'repetitious', 'repetitive', 'rephrase', 'replace', 'replaced', 'replacement', 'replaces', 'replacing', 'replay', 'replayable', 'replayed', 'replaying', 'replays', 'replete', 'replica']
Every 700th feature:
['00', 'affections', 'appropriately', 'barbra', 'blurbs', 'butchered', 'cheese', 'commitment', 'courts', 'deconstructed', 'disgraceful', 'dvds', 'eschews', 'fell', 'freezer', 'goriest', 'hauser', 'hungary', 'insinuate', 'juggle', 'leering', 'maelstrom', 'messiah', 'music', 'occasional', 'parking', 'pleasantville', 'pronunciati

위에서 볼 수 있듯이 최소 문서 개수를 설정하는 것만으로도 희귀한 단어와 철자가 틀린 단어들이 사라진것을 확인 할 수 있다.

In [27]:
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)
print("최상의 교차 검증 점수: {:.2f}".format(grid.best_score_))

최상의 교차 검증 점수: 0.89


## 7.4 불용어(Stop-Words)
불용어는 영어에서 `and, is, there,...`나 한글에서 `은, 는, 이, 가,...`등과 같이 텍스트 분석에 있어 의미를 가지지 않는 유용하지 않은 단어들을 제외하는 것을 말한다. scikit-learn에서는 `feature_extraction.text` 모듈에 영어의 불용어를 가지고 있다.

In [30]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

print("불용어 개수: {}".format(len(ENGLISH_STOP_WORDS)))
print("매 10번째 불용어:\n{}".format(list(ENGLISH_STOP_WORDS)[::10]))

불용어 개수: 318
매 10번째 불용어:
['down', 'anyhow', 'anyway', 'beside', 'although', 'six', 'but', 'anyone', 'whereupon', 'rather', 'herein', 'couldnt', 'sixty', 'part', 'who', 'latter', 'mostly', 'few', 'us', 'see', 'co', 'former', 'i', 'such', 'empty', 'be', 'amongst', 'almost', 'thin', 'may', 'either', 'get']


In [33]:
vect = CountVectorizer(min_df=5, stop_words="english").fit(text_train)
X_train = vect.transform(text_train)
print("불용어가 제거된 X_train:\n{}".format(repr(X_train)))

불용어가 제거된 X_train:
<25000x26966 sparse matrix of type '<class 'numpy.int64'>'
	with 2149958 stored elements in Compressed Sparse Row format>


In [34]:
grid = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid.fit(X_train, y_train)
print("최상의 크로스 밸리데이션 점수: {:.2f}".format(grid.best_score_))

최상의 크로스 밸리데이션 점수: 0.88


이번에는 `max_df`를 사용해서 자주 나타나는 단어를 제거한 뒤 테스트를 해보자.

In [36]:
# from sklearn.pipeline import make_pipeline
# pipe = make_pipeline(CountVectorizer(), LogisticRegression())
# param_grid = {'countvectorizer__max_df': [100, 1000, 10000, 20000], 'logisticregression__C': [0.001, 0.01, 0.1, 1, 10]}
# grid = GridSearchCV(pipe, param_grid, cv=5)
# grid.fit(text_train, y_train)
# print("최상의 크로스 밸리데이션 점수: {:.2f}".format(grid.best_score_))
# print(grid.best_params_)

## 7.5 TF-IDF 로 데이터 스케일 변경하기
TF-IDF는 정보 검색(Information Retrieval)과 텍스트 마이닝에서 사용하는 가중치이다.
여러 문서로 이루어진 문서군이 있을 때 어떤 단어가 특정 문서 내에서 얼마나 중요한 것인지를 나타내는 통계적 수치를 말한다.
- **TF(Term Frequency)**: 단어빈도로 특정 단어가 문서 내에 얼만큼의 빈도로 등장하는지를 나타내는 척도
- **IDF(Inverse Document Frequency)**: 역문헌 빈도수로 문서 빈도의 역수. 전체 문서 개수를 해당 단어가 포함된 문서의 개수로 나눈 것을 의미 

Scikit-learㅇn에서는 `TfidfTransformer`와 `TfidfVectorizer` 이렇게 두 개의 클래스에 TF-IDF가 구현되어 있다.
- `TfidfTransformer`는 `ConterVectorizer`가 만든 희소행렬을 입력받아 변환한다.
- `TfidfVectorizer`는 텍스트 데이터를 입력받아 BOW와 tf-idf 변환을 수행한다.
- `TfidfTransformer`와 `TfidfVectorizer` 둘 다 계산식은 아래와 같다.
$$tfidf(w,d)=tf(\log(\frac{N+1}{N_w + 1})+1)$$

- `TfidfTransformer`와 `TfidfVectorizer` 두 클래스 모두 tf-idf 계산 후 L2 정규화(L2 normalization)을 적용한다. 따라서 유클리드 놈(euclidean norm)이 1이 되도록 각 문서 벡터의 스케일을 바꿔준다. 이렇게 되면 문서의 길이(단어의 수)에 영향을 받지 않게 된다.

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(TfidfVectorizer(min_df=5), LogisticRegression())
param_grid = {'logisticregression__C': [0.01, 0.1, 1]}

grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(text_train, y_train)
print('최상의 CV 점수: {:.2f}'.format(grid.best_score_))

최상의 CV 점수: 0.89


In [39]:
vectorizer = grid.best_estimator_.named_steps["tfidfvectorizer"]
# 훈련 데이터 셋을 변환한다.
X_train = vectorizer.transform(text_train)
# 특성별로 가장 큰 값을 찾는다.
max_value = X_train.max(axis=0).toarray().ravel()
sorted_by_tfidf = max_value.argsort()
# 특성 이름을 구한다.
feature_names = np.array(vectorizer.get_feature_names())

print("가장 낮은 tfidf를 가진 특성:\n{}".format(
      feature_names[sorted_by_tfidf[:20]]))

print("가장 높은 tfidf를 가진 특성: \n{}".format(
      feature_names[sorted_by_tfidf[-20:]]))

가장 낮은 tfidf를 가진 특성:
['suplexes' 'gauche' 'hypocrites' 'oncoming' 'songwriting' 'galadriel'
 'emerald' 'mclaughlin' 'sylvain' 'oversee' 'cataclysmic' 'pressuring'
 'uphold' 'thieving' 'inconsiderate' 'ware' 'denim' 'reverting' 'booed'
 'spacious']
가장 높은 tfidf를 가진 특성: 
['gadget' 'sucks' 'zatoichi' 'demons' 'lennon' 'bye' 'dev' 'weller'
 'sasquatch' 'botched' 'xica' 'darkman' 'woo' 'casper' 'doodlebops'
 'smallville' 'wei' 'scanners' 'steve' 'pokemon']


In [40]:
sorted_by_idf = np.argsort(vectorizer.idf_)
print("가장 낮은 idf를 가진 특성:\n{}".format(
       feature_names[sorted_by_idf[:100]]))

가장 낮은 idf를 가진 특성:
['the' 'and' 'of' 'to' 'this' 'is' 'it' 'in' 'that' 'but' 'for' 'with'
 'was' 'as' 'on' 'movie' 'not' 'have' 'one' 'be' 'film' 'are' 'you' 'all'
 'at' 'an' 'by' 'so' 'from' 'like' 'who' 'they' 'there' 'if' 'his' 'out'
 'just' 'about' 'he' 'or' 'has' 'what' 'some' 'good' 'can' 'more' 'when'
 'time' 'up' 'very' 'even' 'only' 'no' 'would' 'my' 'see' 'really' 'story'
 'which' 'well' 'had' 'me' 'than' 'much' 'their' 'get' 'were' 'other'
 'been' 'do' 'most' 'don' 'her' 'also' 'into' 'first' 'made' 'how' 'great'
 'because' 'will' 'people' 'make' 'way' 'could' 'we' 'bad' 'after' 'any'
 'too' 'then' 'them' 'she' 'watch' 'think' 'acting' 'movies' 'seen' 'its'
 'him']
